In [1]:
#written on 1.21.21 by @caramnix 
#modified/updated on 2.26.22 
#!pip install xmltodict
import os, json
import xmltodict
import numpy as np
import pandas as pd

In [2]:
#uses data from https://www.govinfo.gov/bulkdata/BILLSTATUS/116/hr
#user guide for data: https://github.com/usgpo/bill-status/blob/master/BILLSTATUS-XML_User_User-Guide.md
os.chdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress")

In [3]:
pwd()

'/Users/caranix/Library/CloudStorage/OneDrive-TheOhioStateUniversity/Social Influence/Data/Congress'

In [6]:
wd= "/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress"

In [7]:
#create directories for storing .jsons

#116 House
#os.mkdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/House/BILLSTATUS-116-hr-JSON/")
#os.mkdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/House/BILLSTATUS-116-hres-JSON/")

#115 House
#os.mkdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/115/House/BILLSTATUS-115-hres-JSON/")
#os.mkdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/115/House/BILLSTATUS-115-hr-JSON/")

#116 Senate
#os.mkdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/Senate/BILLSTATUS-116-s-JSON/")
#os.mkdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/Senate/BILLSTATUS-116-sres-JSON/")

#115 Senate
os.mkdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/115/Senate/BILLSTATUS-115-s-JSON/")
os.mkdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/115/Senate/BILLSTATUS-115-sres-JSON/")
        
         

In [17]:
#code to convert XML to JSON because I'm picky about my data types mkay 

#s, and sres 
#note: changed paths accordingly
path = wd+ "/115/Senate/BILLSTATUS-115-sres"
jsonp = wd+ "/115/Senate/BILLSTATUS-115-sres-JSON/"

for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    with open(fullname, 'r') as f:
        xmlString = f.read()
        json_output =  jsonp+ filename[:-4] +".json"
        #print(json_output)
    with open(json_output, 'w') as f:
        json.dump(xmltodict.parse(xmlString), f, indent=4)

In [10]:
#code to reformat data into a managable format, pulling only the information needed from the .json
#the info being, for each bill, it's sponsor (and their last name, unique id and party), cosponsors(and their last name(s), unique id(s) and party)
#Also the type of bill and the bill number (which is saved as the key, but as a precaution als including it in the dict)

def get_dict_for_bill(bill, id_numbers):
    try: 
        sponser_full_name= bill['billStatus']['bill']['sponsors']['item']['fullName']
        sponser_last_name= bill['billStatus']['bill']['sponsors']['item']['lastName']
        sponser_id= bill['billStatus']['bill']['sponsors']['item']['identifiers']["lisID"]
        id_numbers.append(int(sponser_id))
        sponser_party= bill['billStatus']['bill']['sponsors']['item']['party']
        if bill['billStatus']['bill']['policyArea'] is None: #some bills do not have a type. 
            type_bill = "not given"
        else: 
            type_bill = bill['billStatus']['bill']['policyArea']['name']
        bill_id = bill['billStatus']['bill']['billNumber']
        cospon_dict= {}
        if bill['billStatus']['bill']['cosponsors'] is None: #some bills have no cosponsors!
            cospon_dict ={} 
        else: 
            n_cospon= len(bill['billStatus']['bill']['cosponsors']['item'])

            for i in range(0,n_cospon): 
                try:
                    cospon_fname = bill['billStatus']['bill']['cosponsors']['item'][i]['fullName']
                    cospon_lname = bill['billStatus']['bill']['cosponsors']['item'][i]['lastName']
                    cospon_party = bill['billStatus']['bill']['cosponsors']['item'][i]['party']
                    cospon_id = bill['billStatus']['bill']['cosponsors']['item'][i]['identifiers']['lisID']
                    id_numbers.append(int(cospon_id)) #inorder to have all the id numbers of congresspeople
                    cospon_dict[i]= {"LAST NAME" : cospon_lname, "ID" : cospon_id, "PARTY" : cospon_party, "FULL NAME": cospon_fname}
                except KeyError: #see note below 
                    cospon_fname = bill['billStatus']['bill']['cosponsors']['item']['fullName']
                    cospon_lname = bill['billStatus']['bill']['cosponsors']['item']['lastName']
                    cospon_party = bill['billStatus']['bill']['cosponsors']['item']['party']
                    cospon_id = bill['billStatus']['bill']['cosponsors']['item']['identifiers']['lisID']
                    id_numbers.append(int(cospon_id))
                    cospon_dict[0]= {"LAST NAME" : cospon_lname, "ID" : cospon_id, "PARTY" : cospon_party,"FULL NAME": cospon_fname}

        #returning a dictionary of all info collected from bill
        return({"sponser": {"LAST NAME" : sponser_last_name, "ID" : sponser_id, "PARTY" : sponser_party, "FULL NAME" : sponser_full_name}, 
                        "cosponsers": cospon_dict, 
                        "type" : type_bill, 
                        "bill number": bill_id,
                        "action code": check_action(bill)} #new addition 
              )

    except TypeError: 
        print(bill['billStatus']['bill']['billNumber'])
        pass
        



#note: okay, so the data is formatted weirdly, in that if there is only one cosponsor, 
#instead of sticking with how they've been formatting it they instead don't tie 0:cosponsor,
#and instead tie item: cosponsor, this catches that error and fixes it. 



In [11]:
def check_action(bill): 
    num_actions= len(bill['billStatus']['bill']['actions']['item'])
    action_code_fin ="00000"
    for i in range(0, num_actions): 
        try: 
            action_code= bill['billStatus']['bill']['actions']['item'][i]["actionCode"]
        except KeyError: 
            action_code= "00000"
            pass 
        if (action_code== "8000" or action_code== "17000" or action_code== "36000"): 
            action_code_fin= action_code
    return(action_code_fin)

In [202]:
#make dict for HR-116 
os.chdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/House")

In [203]:
Bill_Dict={} 
id_nums = [] 
bill_nums = []

path = "BILLSTATUS-116-hr-JSON"
for filename in os.listdir(path): #go through the folder at the path given and for all the jsons, get the cleaned bill dictionaru
    if not filename.endswith('.json'): continue
    fullname = os.path.join(path, filename)
    with open(fullname, 'r') as read_file:
        bill_of_interest = json.load(read_file)
        bill_id = bill_of_interest['billStatus']['bill']['billNumber']
        bill_nums.append(bill_id) #also store bill ids 
        Bill_Dict[bill_id]= get_dict_for_bill(bill_of_interest,id_nums)


In [204]:
len(Bill_Dict) #sanity check, we passed in 9062 bills and we got that many keys in our resulting dict! 


9062

In [86]:
#save our Bill Dictionary as a json!
#with open('BILLSTATUS-116-hr-JSON-dict.json', 'w') as fp:
#    json.dump(Bill_Dict, fp)

In [32]:
#now can read in easily! HR 116 Data 
#os.chdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/House")
#f = open('BILLSTATUS-116-hr-JSON-dict.json')
#Bill_Dict = json.load(f)

In [46]:
#len(Bill_Dict)

In [ ]:
#make dict for HRES-116 

In [205]:
os.chdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/House")

Bill_Dict2={} 
id_nums2 = [] 
bill_nums2 = []

path = "BILLSTATUS-116-hres-JSON"
for filename in os.listdir(path): #go through the folder at the path given and for all the jsons, get the cleaned bill dictionaru
    if not filename.endswith('.json'): continue
    fullname = os.path.join(path, filename)
    with open(fullname, 'r') as read_file:
        bill_of_interest = json.load(read_file)
        bill_id = bill_of_interest['billStatus']['bill']['billNumber']
        bill_nums2.append(bill_id) #also store bill ids 
        Bill_Dict2[bill_id+"HRES"]= get_dict_for_HRESbill(bill_of_interest,id_nums2)

765


In [21]:
#COMBINE THEM
def Merge(dict1, dict2):
    return(dict2.update(dict1))

In [207]:
Merge(Bill_Dict2,Bill_Dict) #merge all into Bill_Dict!

In [208]:
len(Bill_Dict)

10335

In [ ]:
#with open('BILLSTATUS-116-hr-and-hres-JSON-dict.json', 'w') as fp:
#    json.dump(Bill_Dict, fp)

In [43]:
#os.chdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/House")
#f = open('BILLSTATUS-116-hr-and-hres-JSON-dict.json')
#Bill_Dict = json.load(f)

In [210]:
master_id_nums= id_nums + id_nums2
len(master_id_nums)
u_id_nums=np.unique(master_id_nums)

In [70]:
u_id_nums=np.unique(master_id_nums)

In [71]:
len(u_id_nums) #448 - 435 + people getting replaced 

448

In [72]:
len(Bill_Dict)

10335

In [211]:
Bill_Dict.pop('765HRES') #HAD TO CUT- problematic

In [212]:
#count number cosponsors/sponsors
columns= ["Number of Bills Cosponsored", "Number of Bills Sponsered", "ID"]
df_ = pd.DataFrame(index=u_id_nums, columns= columns)
df_ = df_.fillna(0)

for key in Bill_Dict: 
    sponser_id= Bill_Dict[str(key)]['sponser']['ID']
    df_.loc[int(sponser_id)]["ID"] = sponser_id
    df_.loc[int(sponser_id)]["Number of Bills Sponsered"] += 1
    num_cospo= len(Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        df_.loc[int(cospo_id)]["Number of Bills Cosponsored"] += 1
   

In [213]:
df_.loc[df_['ID'] == 0] #print out those w/ 0 sponsorships 

,Number of Bills Cosponsored,Number of Bills Sponsered,ID
798,51,0,0
2053,9,0,0
2479,35,0,0


In [214]:
df_.loc[798]["ID"]= "798"
df_.loc[2053]["ID"]= "2053"
df_.loc[2479]["ID"]= "2479"

In [ ]:
#HELPFUL EXAMPLE
adj_test = pd.DataFrame(index=["1", "2", "3"], columns= ["1", "2", "3"])
adj_test = adj_test.fillna(0)
adj_test.loc["1"]["3"] += 1 # 1 cospoed 3
adj_test.loc["2"]["3"] += 1 # 2 cospoed 3
print(adj_test)
print(adj_test.sum(axis=0)) #3 was cosponsored 2 times 
print(adj_test.sum(axis=1)) #1 cosponsored one time, #2 cosponsoed one time 

In [215]:
#FOWLER VERSION OF COSPONSORSHIP 
#attach cosponsors to sponsors in directed fashion 
adjf_ = pd.DataFrame(index=u_id_nums, columns= u_id_nums)
adjf_ = adjf_.fillna(0)
for key in Bill_Dict: 
    sponser_id= Bill_Dict[str(key)]['sponser']['ID']
    num_cospo= len(Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        adjf_.loc[int(cospo_id)][int(sponser_id)] += 1

In [216]:
pwd()

'/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/House'

In [217]:
adjf_.to_excel("adj_matrix_116_House_Fowler.xlsx")  

In [121]:
adjf_.sum(axis=0) # number of times they were cosponsored by others 

91        74
99      1666
165      113
186      218
208      595
        ... 
2473      48
2479       0
2480       8
2481       1
2483       2
Length: 448, dtype: int64

In [122]:
adjf_.sum(axis=1) # number of times they cosponsored legislation

91      465
99      906
165     205
186     303
208     140
       ... 
2473    123
2479     35
2480     87
2481     29
2483     12
Length: 448, dtype: int64

In [80]:
#make helpful data frame from dict that can be joined 
#congressperson info
data1= {}
for key in Bill_Dict: 
    sponser_id= Bill_Dict[str(key)]['sponser']['ID']   
    if sponser_id not in data1.keys(): 
        data1[int(sponser_id)]= [Bill_Dict[str(key)]['sponser']['LAST NAME'].upper(), Bill_Dict[str(key)]['sponser']['PARTY'], Bill_Dict[str(key)]['sponser']['FULL NAME'], sponser_id]
    num_cospo= len(Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        if cospo_id not in data1.keys():
            data1[int(cospo_id)]= [Bill_Dict[str(key)]['cosponsers'][i]['LAST NAME'].upper(), Bill_Dict[str(key)]['cosponsers'][i]['PARTY'],Bill_Dict[str(key)]['cosponsers'][i]['FULL NAME'], cospo_id]
datf= pd.DataFrame.from_dict(data1, orient='index')
datf.columns= ['Last Name', 'Party', 'Full Name', 'ID'] 
datf["ID"]= pd.to_numeric(datf["ID"])
datf.merge(df_, left_on='ID', right_on='ID') 

In [81]:
datf= pd.DataFrame.from_dict(data1, orient='index')
datf.columns= ['Last Name', 'Party', 'Full Name', 'ID'] 

In [92]:
datf["ID"]= pd.to_numeric(datf["ID"])

In [126]:
datf.merge(df_, left_on='ID', right_on='ID') 

,Last Name,Party,Full Name,ID,Number of Bills Cosponsored,Number of Bills Sponsered
0,ROUDA,D,"Rep. Rouda, Harley [D-CA-48]",2382,632,26
1,GOLDEN,D,"Rep. Golden, Jared F. [D-ME-2]",2469,266,24
2,ROSE,D,"Rep. Rose, Max [D-NY-11]",2426,495,31
3,ADAMS,D,"Rep. Adams, Alma S. [D-NC-12]",2201,407,34
4,WALKER,R,"Rep. Walker, Mark [R-NC-6]",2255,176,17
...,...,...,...,...,...,...
443,HALL,D,"Rep. Hall, Kwanza [D-GA-5]",2483,12,5
444,HOLLINGSWORTH,R,"Rep. Hollingsworth, Trey [R-IN-9]",2327,75,8
445,MARINO,R,"Rep. Marino, Tom [R-PA-12]",2053,9,0
446,HOYER,D,"Rep. Hoyer, Steny H. [D-MD-5]",566,59,18


In [127]:
datf.to_csv("leginfo_116H.csv") #save

In [4]:
#make dict for S-116 
os.chdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/Senate")

In [129]:
#make dict for S-116 
os.chdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/Senate")

Senate_Bill_Dict={} 
Senate_id_nums = [] 
Senate_bill_nums = []

path = "BILLSTATUS-116-s-JSON"
for filename in os.listdir(path): #go through the folder at the path given and for all the jsons, get the cleaned bill dictionaru
    if not filename.endswith('.json'): continue
    fullname = os.path.join(path, filename)
    with open(fullname, 'r') as read_file:
        bill_of_interest = json.load(read_file)
        bill_id = bill_of_interest['billStatus']['bill']['billNumber']
        Senate_bill_nums.append(bill_id) #also store bill ids 
        Senate_Bill_Dict[bill_id]= get_dict_for_bill(bill_of_interest,Senate_id_nums)


In [130]:
len(Senate_Bill_Dict)

5086

In [132]:
#save our Bill Dictionary as a json!
with open('BILLSTATUS-116-s-JSON-dict.json', 'w') as fp:
    json.dump(Senate_Bill_Dict, fp)

In [ ]:
#make dict for SRES-116

In [133]:
Senate_Bill_Dict2={} 
Senate_id_nums2 = [] 
Senate_bill_nums2 = []

path = "BILLSTATUS-116-sres-JSON"
for filename in os.listdir(path): #go through the folder at the path given and for all the jsons, get the cleaned bill dictionaru
    if not filename.endswith('.json'): continue
    fullname = os.path.join(path, filename)
    with open(fullname, 'r') as read_file:
        bill_of_interest = json.load(read_file)
        bill_id = bill_of_interest['billStatus']['bill']['billNumber']
        Senate_bill_nums2.append(bill_id) #also store bill ids 
        Senate_Bill_Dict2[bill_id+ 'SRES']= get_dict_for_bill(bill_of_interest,Senate_id_nums2)

In [134]:
len(Senate_Bill_Dict2)

808

In [137]:
with open('BILLSTATUS-116-sres-JSON-dict.json', 'w') as fp:
    json.dump(Senate_Bill_Dict2, fp)

In [138]:
Merge(Senate_Bill_Dict2,Senate_Bill_Dict) 

In [139]:
len(Senate_Bill_Dict)

5894

In [140]:
master_id_nums_senate= Senate_id_nums2 + Senate_id_nums
u_id_nums_sen=np.unique(master_id_nums_senate)

In [142]:
len(u_id_nums_sen)

102

In [143]:
#count number cosponsors/sponsors
columns= ["Number of Bills Cosponsored", "Number of Bills Sponsered", "ID"]
df_S = pd.DataFrame(index=u_id_nums_sen, columns= columns)
df_S = df_S.fillna(0)

for key in Senate_Bill_Dict: 
    sponser_id= Senate_Bill_Dict[str(key)]['sponser']['ID']
    df_S.loc[int(sponser_id)]["ID"] = sponser_id
    df_S.loc[int(sponser_id)]["Number of Bills Sponsered"] += 1
    num_cospo= len(Senate_Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Senate_Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        df_S.loc[int(cospo_id)]["Number of Bills Cosponsored"] += 1
   

In [148]:
df_S.loc[2482]["ID"]= "2482"

In [144]:
#FOWLER VERSION OF COSPONSORSHIP 
#attach cosponsors to sponsors in directed fashion 
adjf_S = pd.DataFrame(index=u_id_nums_sen, columns= u_id_nums_sen)
adjf_S = adjf_S.fillna(0)
for key in Senate_Bill_Dict: 
    sponser_id= Senate_Bill_Dict[str(key)]['sponser']['ID']
    num_cospo= len(Senate_Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Senate_Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        adjf_S.loc[int(cospo_id)][int(sponser_id)] += 1

In [149]:
adjf_S

,136,153,172,174,179,250,326,452,457,583,...,2339,2364,2365,2366,2462,2464,2466,2470,2474,2482
136,0,4,2,20,4,3,31,5,9,4,...,2,4,9,0,2,0,0,1,0,0
153,3,0,0,1,1,1,1,9,1,4,...,0,1,0,0,2,0,0,0,0,0
172,7,2,0,11,3,2,8,5,2,1,...,2,1,1,1,1,0,0,0,0,0
174,13,5,4,0,6,5,19,7,2,3,...,2,2,6,1,0,1,0,0,0,0
179,8,2,2,12,0,1,10,6,5,2,...,3,1,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,1,2,1,3,0,1,1,10,3,3,...,2,1,0,1,2,0,0,5,0,0
2466,0,1,0,3,0,1,3,7,3,1,...,3,0,0,0,0,1,0,0,0,0
2470,2,1,2,5,1,1,2,11,5,3,...,5,2,1,2,9,8,0,0,2,0
2474,0,0,0,2,0,1,0,7,3,4,...,1,0,0,2,10,6,0,5,0,0


In [150]:
adjf_S.to_excel("adj_matrix_116_Senate_Fowler.xlsx")  

In [151]:
#make helpful data frame from dict that can be joined 
#congressperson info
data1= {}
for key in Senate_Bill_Dict: 
    sponser_id= Senate_Bill_Dict[str(key)]['sponser']['ID']   
    if sponser_id not in data1.keys(): 
        data1[int(sponser_id)]= [Senate_Bill_Dict[str(key)]['sponser']['LAST NAME'].upper(), Senate_Bill_Dict[str(key)]['sponser']['PARTY'], Senate_Bill_Dict[str(key)]['sponser']['FULL NAME'], sponser_id]
    num_cospo= len(Senate_Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Senate_Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        if cospo_id not in data1.keys():
            data1[int(cospo_id)]= [Senate_Bill_Dict[str(key)]['cosponsers'][i]['LAST NAME'].upper(), Senate_Bill_Dict[str(key)]['cosponsers'][i]['PARTY'],Senate_Bill_Dict[str(key)]['cosponsers'][i]['FULL NAME'], cospo_id]



In [152]:
datf= pd.DataFrame.from_dict(data1, orient='index')
datf.columns= ['Last Name', 'Party', 'Full Name', 'ID'] 

In [153]:
datf["ID"]= pd.to_numeric(datf["ID"])

In [155]:
datf.merge(df_S, left_on='ID', right_on='ID') 

,Last Name,Party,Full Name,ID,Number of Bills Cosponsored,Number of Bills Sponsered
0,KLOBUCHAR,D,"Sen. Klobuchar, Amy [D-MN]",1826,807,118
1,CASEY,D,"Sen. Casey, Robert P., Jr. [D-PA]",1828,531,115
2,CAPITO,R,"Sen. Capito, Shelley Moore [R-WV]",1676,415,29
3,ERNST,R,"Sen. Ernst, Joni [R-IA]",2283,371,52
4,ROSEN,D,"Sen. Rosen, Jacky [D-NV]",2339,573,37
...,...,...,...,...,...,...
97,BURR,R,"Sen. Burr, Richard [R-NC]",153,135,34
98,PAUL,R,"Sen. Paul, Rand [R-KY]",2082,104,63
99,JOHNSON,R,"Sen. Johnson, Ron [R-WI]",2086,122,43
100,KELLY,D,"Sen. Kelly, Mark [D-AZ]",2482,28,0


In [156]:
datf.to_csv("leginfo_116S.csv") #save

In [ ]:
## NOW DO 115

In [157]:
#make dict for HR-116 
os.chdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/115/House")

In [163]:
Bill_Dict={} 
id_nums = [] 
bill_nums = []

path = "BILLSTATUS-115-hr-JSON"
for filename in os.listdir(path): #go through the folder at the path given and for all the jsons, get the cleaned bill dictionaru
    if not filename.endswith('.json'): continue
    fullname = os.path.join(path, filename)
    with open(fullname, 'r') as read_file:
        bill_of_interest = json.load(read_file)
        bill_id = bill_of_interest['billStatus']['bill']['billNumber']
        bill_nums.append(bill_id) #also store bill ids 
        Bill_Dict[bill_id]= get_dict_for_bill(bill_of_interest,id_nums)


In [164]:
len(Bill_Dict)

7394

In [165]:
os.chdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/115/House")

Bill_Dict2={} 
id_nums2 = [] 
bill_nums2 = []

path = "BILLSTATUS-115-hres-JSON"
for filename in os.listdir(path): #go through the folder at the path given and for all the jsons, get the cleaned bill dictionaru
    if not filename.endswith('.json'): continue
    fullname = os.path.join(path, filename)
    with open(fullname, 'r') as read_file:
        bill_of_interest = json.load(read_file)
        bill_id = bill_of_interest['billStatus']['bill']['billNumber']
        bill_nums2.append(bill_id) #also store bill ids 
        Bill_Dict2[bill_id+"HRES"]= get_dict_for_HRESbill(bill_of_interest,id_nums2)

In [166]:
Merge(Bill_Dict2,Bill_Dict) #merge all into Bill_Dict!

In [185]:
len(Bill_Dict)

8581

In [186]:
#save our Bill Dictionary as a json!
with open('BILLSTATUS-115-hr-and-hres-JSON.json', 'w') as fp:
    json.dump(Bill_Dict, fp)

In [168]:
master_id_nums= id_nums + id_nums2
len(master_id_nums)
u_id_nums=np.unique(master_id_nums)

In [169]:
#count number cosponsors/sponsors
columns= ["Number of Bills Cosponsored", "Number of Bills Sponsered", "ID"]
df_ = pd.DataFrame(index=u_id_nums, columns= columns)
df_ = df_.fillna(0)

for key in Bill_Dict: 
    sponser_id= Bill_Dict[str(key)]['sponser']['ID']
    df_.loc[int(sponser_id)]["ID"] = sponser_id
    df_.loc[int(sponser_id)]["Number of Bills Sponsered"] += 1
    num_cospo= len(Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        df_.loc[int(cospo_id)]["Number of Bills Cosponsored"] += 1
   

In [173]:
df_.loc[df_['ID'] == 0] #print out those w/ 0 sponsorships 

,Number of Bills Cosponsored,Number of Bills Sponsered,ID


In [172]:
df_.loc[70]["ID"]= "70"
df_.loc[2155]["ID"]= "2155"
df_.loc[2254]["ID"]= "2254"
df_.loc[2369]["ID"]= "2369"
df_.loc[2370]["ID"]= "2370"
df_.loc[2372]["ID"]= "2372"

In [174]:
#FOWLER VERSION OF COSPONSORSHIP 
#attach cosponsors to sponsors in directed fashion 
adjf_ = pd.DataFrame(index=u_id_nums, columns= u_id_nums)
adjf_ = adjf_.fillna(0)
for key in Bill_Dict: 
    sponser_id= Bill_Dict[str(key)]['sponser']['ID']
    num_cospo= len(Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        adjf_.loc[int(cospo_id)][int(sponser_id)] += 1

In [177]:
df_

,Number of Bills Cosponsored,Number of Bills Sponsered,ID
62,110,7,62
70,2,0,70
91,334,4,91
99,770,31,99
165,171,16,165
...,...,...,...
2371,45,1,2371
2372,2,0,2372
2373,14,1,2373
2374,13,1,2374


In [175]:
adjf_

,62,70,91,99,165,186,208,229,231,256,...,2363,2367,2368,2369,2370,2371,2372,2373,2374,2375
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,1,0,0,1,0,0,2,3,2,1,...,0,0,0,0,0,0,0,0,0,0
99,1,0,1,0,1,0,2,5,0,1,...,0,0,0,0,0,0,0,0,0,0
165,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2372,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2373,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2374,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [179]:
adjf_.to_excel("adj_matrix_115_House_Fowler.xlsx")  

In [180]:
#make helpful data frame from dict that can be joined 
#congressperson info
data1= {}
for key in Bill_Dict: 
    sponser_id= Bill_Dict[str(key)]['sponser']['ID']   
    if sponser_id not in data1.keys(): 
        data1[int(sponser_id)]= [Bill_Dict[str(key)]['sponser']['LAST NAME'].upper(), Bill_Dict[str(key)]['sponser']['PARTY'], Bill_Dict[str(key)]['sponser']['FULL NAME'], sponser_id]
    num_cospo= len(Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        if cospo_id not in data1.keys():
            data1[int(cospo_id)]= [Bill_Dict[str(key)]['cosponsers'][i]['LAST NAME'].upper(), Bill_Dict[str(key)]['cosponsers'][i]['PARTY'],Bill_Dict[str(key)]['cosponsers'][i]['FULL NAME'], cospo_id]
datf= pd.DataFrame.from_dict(data1, orient='index')
datf.columns= ['Last Name', 'Party', 'Full Name', 'ID'] 
datf["ID"]= pd.to_numeric(datf["ID"])
datf.merge(df_, left_on='ID', right_on='ID') 

,Last Name,Party,Full Name,ID,Number of Bills Cosponsored,Number of Bills Sponsered
0,TIPTON,R,"Rep. Tipton, Scott R. [R-CO-3]",1997,302,37
1,POLIS,D,"Rep. Polis, Jared [D-CO-2]",1910,491,30
2,THOMPSON,D,"Rep. Thompson, Mike [D-CA-5]",1593,353,24
3,AMODEI,R,"Rep. Amodei, Mark E. [R-NV-2]",2090,168,14
4,BURGESS,R,"Rep. Burgess, Michael C. [R-TX-26]",1751,166,46
...,...,...,...,...,...,...
448,WILD,D,"Rep. Wild, Susan [D-PA-15]",2374,13,1
449,BECERRA,D,"Rep. Becerra, Xavier [D-CA-34]",70,2,0
450,JONES,D,"Rep. Jones, Brenda [D-MI-13]",2375,4,2
451,ZINKE,R,"Rep. Zinke, Ryan K. [R-MT-At Large]",2254,2,0


In [200]:
datf.to_csv("leginfo_115H.csv") #save

In [ ]:
#make dict for Senate 115 


In [12]:
#make dict for S-115
os.chdir("/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/115/Senate")

Senate_Bill_Dict={} 
Senate_id_nums = [] 
Senate_bill_nums = []

path = "BILLSTATUS-115-s-JSON"
for filename in os.listdir(path): #go through the folder at the path given and for all the jsons, get the cleaned bill dictionaru
    if not filename.endswith('.json'): continue
    fullname = os.path.join(path, filename)
    with open(fullname, 'r') as read_file:
        bill_of_interest = json.load(read_file)
        bill_id = bill_of_interest['billStatus']['bill']['billNumber']
        Senate_bill_nums.append(bill_id) #also store bill ids 
        Senate_Bill_Dict[bill_id]= get_dict_for_bill(bill_of_interest,Senate_id_nums)


In [13]:
len(Senate_Bill_Dict)

3805

In [14]:
#save our Bill Dictionary as a json!
with open('BILLSTATUS-115-s-JSON-dict.json', 'w') as fp:
    json.dump(Senate_Bill_Dict, fp)

In [18]:
Senate_Bill_Dict2={} 
Senate_id_nums2 = [] 
Senate_bill_nums2 = []

path = "BILLSTATUS-115-sres-JSON"
for filename in os.listdir(path): #go through the folder at the path given and for all the jsons, get the cleaned bill dictionaru
    if not filename.endswith('.json'): continue
    fullname = os.path.join(path, filename)
    with open(fullname, 'r') as read_file:
        bill_of_interest = json.load(read_file)
        bill_id = bill_of_interest['billStatus']['bill']['billNumber']
        Senate_bill_nums2.append(bill_id) #also store bill ids 
        Senate_Bill_Dict2[bill_id+ 'SRES']= get_dict_for_bill(bill_of_interest,Senate_id_nums2)

In [19]:
len(Senate_Bill_Dict2)

747

In [22]:
Merge(Senate_Bill_Dict2,Senate_Bill_Dict) 

In [23]:
len(Senate_Bill_Dict)

4552

In [26]:
with open('BILLSTATUS-115-s-and-sres-JSON-dict.json', 'w') as fp:
    json.dump(Senate_Bill_Dict, fp)

In [24]:
master_id_nums_senate= Senate_id_nums2 + Senate_id_nums
u_id_nums_sen=np.unique(master_id_nums_senate)

In [25]:
#count number cosponsors/sponsors
columns= ["Number of Bills Cosponsored", "Number of Bills Sponsered", "ID"]
df_S = pd.DataFrame(index=u_id_nums_sen, columns= columns)
df_S = df_S.fillna(0)

for key in Senate_Bill_Dict: 
    sponser_id= Senate_Bill_Dict[str(key)]['sponser']['ID']
    df_S.loc[int(sponser_id)]["ID"] = sponser_id
    df_S.loc[int(sponser_id)]["Number of Bills Sponsered"] += 1
    num_cospo= len(Senate_Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Senate_Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        df_S.loc[int(cospo_id)]["Number of Bills Cosponsored"] += 1
   

In [28]:
df_S.loc[df_S['ID'] == 0] 

,Number of Bills Cosponsored,Number of Bills Sponsered,ID
1548,1,0,0


In [30]:
df_S.loc[1548]["ID"]= "1548"

In [27]:
#FOWLER VERSION OF COSPONSORSHIP 
#attach cosponsors to sponsors in directed fashion 
adjf_S = pd.DataFrame(index=u_id_nums_sen, columns= u_id_nums_sen)
adjf_S = adjf_S.fillna(0)
for key in Senate_Bill_Dict: 
    sponser_id= Senate_Bill_Dict[str(key)]['sponser']['ID']
    num_cospo= len(Senate_Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Senate_Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        adjf_S.loc[int(cospo_id)][int(sponser_id)] += 1

In [31]:
adjf_S.to_excel("adj_matrix_115_Senate_Fowler.xlsx")  

In [32]:
#make helpful data frame from dict that can be joined 
#congressperson info
data1= {}
for key in Senate_Bill_Dict: 
    sponser_id= Senate_Bill_Dict[str(key)]['sponser']['ID']   
    if sponser_id not in data1.keys(): 
        data1[int(sponser_id)]= [Senate_Bill_Dict[str(key)]['sponser']['LAST NAME'].upper(), Senate_Bill_Dict[str(key)]['sponser']['PARTY'], Senate_Bill_Dict[str(key)]['sponser']['FULL NAME'], sponser_id]
    num_cospo= len(Senate_Bill_Dict[str(key)]['cosponsers'])
    for i in range(0,num_cospo): 
        cospo_id= Senate_Bill_Dict[str(key)]['cosponsers'][i]["ID"]
        if cospo_id not in data1.keys():
            data1[int(cospo_id)]= [Senate_Bill_Dict[str(key)]['cosponsers'][i]['LAST NAME'].upper(), Senate_Bill_Dict[str(key)]['cosponsers'][i]['PARTY'],Senate_Bill_Dict[str(key)]['cosponsers'][i]['FULL NAME'], cospo_id]

datf= pd.DataFrame.from_dict(data1, orient='index')
datf.columns= ['Last Name', 'Party', 'Full Name', 'ID']
datf["ID"]= pd.to_numeric(datf["ID"])
datf.merge(df_S, left_on='ID', right_on='ID') 



,Last Name,Party,Full Name,ID,Number of Bills Cosponsored,Number of Bills Sponsered
0,SANDERS,I,"Sen. Sanders, Bernard [I-VT]",1010,306,30
1,TOOMEY,R,"Sen. Toomey, Patrick [R-PA]",1594,119,22
2,BLUNT,R,"Sen. Blunt, Roy [R-MO]",1464,249,43
3,CAPITO,R,"Sen. Capito, Shelley Moore [R-WV]",1676,305,25
4,CORNYN,R,"Sen. Cornyn, John [R-TX]",1692,245,80
...,...,...,...,...,...,...
100,CRAPO,R,"Sen. Crapo, Mike [R-ID]",250,183,30
101,CORKER,R,"Sen. Corker, Bob [R-TN]",1825,69,18
102,MCCONNELL,R,"Sen. McConnell, Mitch [R-KY]",1395,55,44
103,KYL,R,"Sen. Kyl, Jon [R-AZ]",655,7,3


In [34]:
datf.to_csv("leginfo_115S.csv") #save

In [140]:
#df_2.to_excel("billinfo_tojoin.xlsx") #save

In [ ]:
#TESTING ON ONE BILL 
#Bill_Dict={} 
#id_nums = [] 
#bill_nums = []

#billpath= "/Users/caranix/OneDrive - The Ohio State University/Social Influence/Data/Congress/116/House/BILLSTATUS-116-hres-JSON/BILLSTATUS-116hres765.json"
#with open(billpath, 'r') as read_file:
 #   bill_of_interest = json.load(read_file)
 ##   bill_id = bill_of_interest['billStatus']['bill']['billNumber']
 #   bill_nums.append(bill_id) #also store bill ids 
  #  Bill_Dict[bill_id +"HRES"]= get_dict_for_HRESbill(bill_of_interest,id_nums)

In [ ]:
#junk code 
#columns= ["Last Name", "Party"]
#df_1 = pd.DataFrame(index=u_id_nums, columns= columns)
#df_1 = df_1.fillna(0)

#for key in Bill_Dict: 
 #   sponser_id= Bill_Dict[str(key)]['sponser']['ID']   
 #   df_1.loc[int(sponser_id)]["Last Name"] = Bill_Dict[str(key)]['sponser']['LAST NAME']
 #   df_1.loc[int(sponser_id)]["Party"] = Bill_Dict[str(key)]['sponser']['PARTY']
 #   num_cospo= len(Bill_Dict[str(key)]['cosponsers'])
  #  for i in range(0,num_cospo): 
  #      cospo_id= Bill_Dict[str(key)]['cosponsers'][i]["ID"]
  #      df_1.loc[int(cospo_id)]["Last Name"] = Bill_Dict[str(key)]['cosponsers'][i]['LAST NAME']
  #      df_1.loc[int(cospo_id)]["Party"] = Bill_Dict[str(key)]['cosponsers'][i]['PARTY']